# **Human Action Recognition with MMPose and Spatio-Tempoal Graph Convolutional Network**
Yolov3 + Resnet152 + STGCN

# Download the Florence 3D action dataset.

## Florence 3D actions dataset

The dataset collected at the University of Florence during 2012, has been captured using a Kinect camera. It includes 9 activities: wave, drink from a bottle, answer phone,clap, tight lace, sit down, stand up, read watch, bow. During acquisition, 10 subjects were asked to perform the above
actions for 2/3 times. This resulted in a total of 215 activity samples.
We suggest a leave-one-actor-out protocol: train your classifier using all the sequences from 9 out of 10 actors and test on the remaining one. Repeat this procedure for all actors and average the 10 classification accuracy values.

Actions 
1.	wave
2.	drink from a bottle
3.	answer phone
4.	clap
5.	tight lace
6.	sit down
7.	stand up
8.	read watch
9.	bow

Videos depicting the actions are named GestureRecording_Id\<ID_GESTURE\>actor\<ID_ACTOR\>idAction\<ID_ACTION\>category\<ID_CATEGORY\>.avi
The file The file Florence_dataset_Features.txt contains all the pose features with annotate actor and actions. Each line is formatted according to the following:

%idvideo idactor idcategory  f1....fn

where f1-f24 are our normalized body part coordinates
and f25 is the normalized frame value.

Specifically:  
  elbows: f1-f6; (1-3 left elbow, 4-6 right elbow, same applies for all other joints)  
  wrists: f13-f18  
  knees: f7-f12  
  ankles: f19-f24  
  normalized frame value: f25  

The file Florence_dataset_WorldCoordinates.txt
Contains the world coordinates for all the joints. Thanks to Maxime Devanne for parsing this data! Each line is formatted according to the following:

%idvideo idactor idcategory  f1....fn
where f1-f45 are world coordinates of all the 15 joints.

Specifically:  
  Head: f1-f3  
  Neck: f4-f6  
  Spine: f7-f9  
  Left Shoulder: f10-f12  
  Left Elbow: f13-f15  
  Left Wrist: f16-f18  
  Right Shoulder: f19-f21  
  Right Elbow: f22-f24  
  Right Wrist: f25-f27  
  Left Hip: f28-f30  
  Left Knee: f31-f33  
  Left Ankle: f34-f36  
  Right Hip: f37-f39  
  Right Knee: f40-f42  
  Right Ankle: f43-f45  


In [ ]:
%%shell
curl https://www.micc.unifi.it/vim/wp-content/uploads/datasets/florence3d_actions.zip -o florence3d_actions.zip
unzip -o -q florence3d_actions.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  303M  100  303M    0     0  14.0M      0  0:00:21  0:00:21 --:--:-- 14.8M


# Functions

In [ ]:
import cv2
from tqdm.notebook import trange, tqdm

def frame_iter(capture, description = ""):
  def _iterator():
    while capture.grab():
      yield capture.retrieve()

  return tqdm(
    _iterator(),
    desc=description,
    total=int(capture.get(cv2.CAP_PROP_FRAME_COUNT)),
    leave=False,
  )


def process_mmdet_results(mmdet_results, cat_id=0):
  """Process mmdet results, and return a list of bboxes.

  :param mmdet_results:
  :param cat_id: category id (default: 0 for human)
  :return: a list of detected bounding boxes
  """
  if isinstance(mmdet_results, tuple):
      det_results = mmdet_results[0]
  else:
      det_results = mmdet_results
  return det_results[cat_id]


# visualization
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

# Installation

## Install MMPose

In [ ]:
%%shell
pip install -q tqdm
# This installation takes a long time due to the compiation of mmcv-full 'about 10mins'
pip install -q mmdet mmcv-full

git clone https://github.com/open-mmlab/mmpose.git
cd mmpose
pip install -q -r requirements.txt
python setup.py -q develop

     |████████████████████████████████| 512kB 10.2MB/s 
     |████████████████████████████████| 225kB 12.0MB/s 
     |████████████████████████████████| 194kB 17.1MB/s 
Cloning into 'mmpose'...
remote: Enumerating objects: 504, done.
remote: Counting objects: 100% (504/504), done.
remote: Compressing objects: 100% (268/268), done.
remote: Total 5856 (delta 317), reused 366 (delta 232), pack-reused 5352
Receiving objects: 100% (5856/5856), 17.57 MiB | 33.56 MiB/s, done.
Resolving deltas: 100% (3779/3779), done.
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 317kB 12.9MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 112kB 14.7MB/s 
     |████████████████████████████████| 7.7MB 14.1MB/s 
     |████████████████████████████████| 4.1MB 42.2MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 71kB 

# Pose estimation using TopDown method

In [ ]:
%%shell
cd /content/
git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 14924, done.
remote: Total 14924 (delta 0), reused 0 (delta 0), pack-reused 14924
Receiving objects: 100% (14924/14924), 16.58 MiB | 26.25 MiB/s, done.
Resolving deltas: 100% (10207/10207), done.


In [ ]:
%cd /content/mmpose
import os
import re
import glob
import pickle
import os.path as osp

/content/mmpose


## Pose estimation definition
PoseResnet152

In [ ]:
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model, vis_pose_result


def inference_pose_estimation_model(video_path, 
                            return_heatmap = True, 
                            save_out_video = True, 
                            out_video_root = '/content/video_results'):
  
  det_model = init_detector(
        '/content/mmdetection/configs/yolo/yolov3_d53_mstrain-416_273e_coco.py', 
        'http://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-416_273e_coco/yolov3_d53_mstrain-416_273e_coco-2b60fcd9.pth', 
        device='cuda:0')
  
  # build the pose model from a config file and a checkpoint file
  pose_model = init_pose_model(
      '/content/mmpose/configs/top_down/rsn/coco/3xrsn50_coco_256x192.py', # model configuration
      'https://download.openmmlab.com/mmpose/top_down/rsn/3xrsn50_coco_256x192-58f57a68_20201127.pth', # pretrained model
      device='cuda:0')

  dataset = pose_model.cfg.data['test']['type']
  cap = cv2.VideoCapture(video_path)

  if save_out_video:
    os.makedirs(out_video_root, exist_ok=True)
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(
        os.path.join(out_video_root,f'vis_{os.path.basename(video_path)}'), fourcc, fps, size)

  # e.g. use ('backbone', ) to return backbone feature
  output_layer_names = ()#('backbone', )
  results = []
  video = frame_iter(cap)
  video.set_postfix({'filename': osp.basename(video_path)})
  for flag, img in video:
    if not flag:
      break

    # test a single image, the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, img)

    # keep the person class bounding boxes.
    person_bboxes = process_mmdet_results(mmdet_results)

    # inference the model
    pose_results, returned_outputs = inference_top_down_pose_model(
      pose_model,
      img,
      person_bboxes,
      bbox_thr=0.3,
      format='xyxy',
      dataset=dataset,
      return_heatmap=return_heatmap,
      outputs=output_layer_names)
    results.append(pose_results)

    if save_out_video:
      # show the results
      vis_img = vis_pose_result(
        pose_model,
        img,
        pose_results,
        dataset=dataset,
        kpt_score_thr=0.3,
        show=False)
      
      videoWriter.write(vis_img)

  video.close()
  cap.release()
  if save_out_video:
      videoWriter.release()
  cv2.destroyAllWindows()
  
  return results

## Run the inference on the Florence 3D action dataset.

In [ ]:
save_out_video = True

video_list = glob.glob('/content/Florence_3d_actions/*.avi')
video_list.sort(reverse=False)
pose_estimation_results = {}
with tqdm(total=len(video_list)) as pbar:
  for video_path in video_list:
    filename = osp.basename(video_path)
    results = inference_pose_estimation_model(video_path, save_out_video = save_out_video)
    pose_estimation_results[filename] = results
    pbar.update(1)
    if save_out_video:
      video_result = video_path.replace('Florence_3d_actions/','video_results/vis_')
      video_result1 = video_result.replace('avi','mp4')
      !ffmpeg -y -loglevel panic -i $video_result -vcodec libx264 $video_result1
      !rm $video_result

In [ ]:
import random
video_result_list = glob.glob('/content/video_results/*.mp4')
video_result = random.choice(video_result_list)
show_local_mp4_video(video_result)

In [ ]:
print(pose_estimation_results.keys())
pickle.dump(pose_estimation_results, open("/content/pose_estimation_results_poseresnet152.pkl", "wb"))

In [ ]:
# Change current working directory to /content
%cd /content
!zip -q -r video_results_poseresnet152.zip video_results

# Action Recognition


In [ ]:
%%shell
cd /content
rm -rf st-gcn-pytorch
git clone https://github.com/taznux/st-gcn-pytorch
cd st-gcn-pytorch
mkdir dataset models
ln -sf /content/Florence_3d_actions dataset/

## STGCN Human Action Recognition on Florence 3D skeletion data

In [ ]:
%cd st-gcn-pytorch
#!python preprocess.py # skeleton conversion

In [ ]:
import random
import pickle
import re
import torch
import numpy as np

data = pose_estimation_results

train = []
valid = []
test  = []
train_label = []
valid_label = []
test_label  = []

vid = 0
for d in data:
  ids = list(map(int, re.findall('\d+', d)))
  gid, tid, aid, cid = ids

  #print(vid, d)
  frames = []
  for frame in data[d]:
    if len(frame) > 0:
      pose = frame[0]
      keypoints = np.array(pose['keypoints'])[:17]
      frames.append(keypoints)
  
  if len(frames) >= 32:
    frames = random.sample(frames, 32)
    frames = torch.from_numpy(np.stack(frames, 0))
  else:
    frames = np.stack(frames, 0)
    xloc = np.arange(frames.shape[0])
    new_xloc = np.linspace(0, frames.shape[0], 32)
    frames = np.reshape(frames, (frames.shape[0], -1)).transpose()

    new_datas = []
    for fr in frames:
      new_datas.append(np.interp(new_xloc, xloc, fr))
    frames = torch.from_numpy(np.stack(new_datas, 0)).t()

  frames = frames.view(32, -1, 3)

  if tid < 9:
    train.append(frames)
    train_label.append(cid-1)
  elif tid < 10:
    valid.append(frames)
    valid_label.append(cid-1)
  else:
    test.append(frames)
    test_label.append(cid-1)

  vid += 1
  
train_label = torch.from_numpy(np.asarray(train_label))
valid_label = torch.from_numpy(np.asarray(valid_label))
test_label  = torch.from_numpy(np.asarray(test_label))

torch.save((torch.stack(train, 0), train_label), '/content/st-gcn-pytorch/dataset/train.pkl')
torch.save((torch.stack(valid, 0), valid_label), '/content/st-gcn-pytorch/dataset/valid.pkl')
torch.save((torch.stack(test, 0),  test_label),  '/content/st-gcn-pytorch/dataset/test.pkl')

In [ ]:
!python main.py --num_epochs 100 --val_step 10 --learning_rate 0.00005 --dropout_rate 0.01 --weight_decay 0.000001 # train and test